# Libraries

##  Remove warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Import libraries

In [ ]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import random
import time
import pandas    as pd
import numpy     as np
from   tqdm      import tqdm


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Visualization library
#
import matplotlib.pyplot   as plt 


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Sklearn library
#
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble      import RandomForestRegressor


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# TabNet library
#
from pytorch_tabnet.tab_model import TabNetRegressor


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#
# User libraries
#
from utils.PerformanceMetrics import RegressionEvaluation
from utils.plot_scatter       import *

# Parameters

## Data handling parameters

In [ ]:
# Parameters
#
filename   = 'metadata/7-kanal-1.csv'

Transformation  = True
Scaling         = 'Standard'

## Neural networks parameters

In [ ]:
Lag        =   12
Horizon    =   6

# Data handling

## Import data


In [ ]:
# Start timer
#
start = time.time()

# Load data
#
df = pd.read_csv( filename )

print('[INFO] Data imported')
print('[INFO] Time: %.2f seconds' % (time.time() - start))

df.head(3)

## Preprocess data

### Set index

In [ ]:
# Convert Date to 'datetime64'
#
df['Date'] = df['Date'].astype('datetime64')

# Set index
#
df.set_index('Date', inplace=True)


df = df.resample('5min').mean().interpolate()
df = pd.DataFrame( df['PM2.5'] )
df.head(3)

In [ ]:
targetSeries = df.columns[-1]

### Split Training/Testing

In [ ]:
idx = int( df.shape[0] * 0.8 )

df_train = df[ :idx ]
df_test  = df[ idx: ]

### Visualization

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(20, 3) )

df_train.plot(ax=ax, color='tab:blue' )
df_test.plot(ax=ax,  color='tab:orange')

plt.legend(['Training', 'Testing'], frameon = False, fontsize = 14)
plt.ylabel(targetSeries, size = 14)
plt.xlabel('Date', size = 14);
plt.xticks(size = 12);
plt.yticks(size = 12);

### Fixing Lag

In [ ]:
df_test = pd.concat([df_train.iloc[-Lag:], df_test])

## Preprocessing

## Data Transformation

In [ ]:
if (Transformation == True):
    
    print('[INFO] Data transformation applied')
    
    VALUE = max(df.min().min(), 1.0)
    
    df_train = np.log( df_train + VALUE)
    df_test  = np.log( df_test  + VALUE)
    
else:
    print('[INFO] No data transformation applied.')  
    
VALUE

In [ ]:
if (Scaling == 'MinMax'):
    print('[INFO] Scaling: MinMax')
    
    # Set scaler
    #
    scaler = MinMaxScaler()

    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
        
elif (Scaling == 'Robust'):
    print('[INFO] Scaling: Robust')
    
    # Set scaler
    #
    scaler = RobustScaler()
     
    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
        
elif (Scaling == 'Standard'):
    print('[INFO] Scaling: Standard')
    
    # Set scaler
    #
    scaler = StandardScaler()

    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
           
else:
    print('[WARNING] Unknown data scaling. Standar scaling was selected')   
    
    # Set scaler
    #
    scaler = StandardScaler()

    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )    

## Create Training/Testing data

In [ ]:
def create_dataset(df, look_back=1, Horizon = 1, SeriesName ='', overlap = 1):
    
    # Check if SeriesName exists in dataset
    #
    if (SeriesName not in df.columns):
        SeriesName = df.columns[-1]
    
    
    date, dataX, dataY = [], [], []

    for i in tqdm( range(0, df.shape[0] + 1  - look_back - Horizon, overlap) ):
                    
        
        data = df[i:(i+look_back+Horizon)].copy()
        # Not sequental interval
        #
        if (data.reset_index().diff()[1:].nunique()['Date'] > 1): continue
         
        # X
        #
        # Lag-Instances from Target Series
#         A = data[ SeriesName ].to_numpy()[:look_back].flatten()        
#         # Information from other features (only from current time)
#         B = data.iloc[look_back-1, data.columns != SeriesName].to_numpy()
#         # Concatenate all information - Instance created
#         dataX.append( np.concatenate([A, B]) )
        
#         dataX.append( data[ SeriesName ].to_numpy()[:look_back].flatten() )



        A = data[ SeriesName ].to_numpy()[:look_back].flatten()        
        # Information from Moving-Average features
        B = np.array([
                       data[:look_back][ SeriesName ].to_numpy()[-4:].mean(),
                       data[:look_back][ SeriesName ].to_numpy()[-7:].mean(),
                       data[:look_back][ SeriesName ].to_numpy().mean(),     
                       #
                       data[:look_back][ SeriesName ].to_numpy()[-4:].std(),
                       data[:look_back][ SeriesName ].to_numpy()[-7:].std(),    
                       #
                       np.sin( data.index.hour[look_back] + data.index.minute[look_back] / 60.0 ),
                       np.cos( data.index.hour[look_back] + data.index.minute[look_back] / 60.0 )
                      ])
        # Concatenate all information - Instance created
        dataX.append( np.concatenate([A, B]) )



        # ALL DATA: dataX.append( data.to_numpy()[:look_back].flatten() )


        # Y
        #
        dataY.append( data[ SeriesName ].to_numpy()[ look_back : look_back + Horizon] )
        
        # Date (ahead) - Needed for visualization
        #
        date.append( data.index[ look_back : look_back + Horizon] )
                      
    return ( np.array(dataX), np.array(dataY), np.array(date) )

In [ ]:
trainX, trainY, _ = create_dataset(df           = df_train, 
                                   look_back    = Lag, 
                                   Horizon      = Horizon, 
                                   SeriesName   = targetSeries,
                                   overlap      = 1,
                                   )

testX,  testY, _  = create_dataset(df           = df_test, 
                                   look_back    = Lag, 
                                   Horizon      = Horizon, 
                                   SeriesName   = targetSeries,
                                   overlap      = Horizon,)



print('Training instances:   %6i' % trainX.shape[0])
print('Testing instances:    %6i' % testX.shape[0])

# Forecasting model: TabNet

## Setup model

In [ ]:
model = TabNetRegressor(verbose = 1, 
                        seed    = 42)

## Training process

In [ ]:
import torch.nn as nn

# Start clock
#
start = time.time()

# Train model
#
model.fit(X_train = trainX, 
          y_train = trainY,
          loss_fn = nn.MSELoss(), # nn.L1Loss(), nn.MSELoss()
          #
          eval_set    = [(testX, testY)],
          batch_size  = 32,
          num_workers = 0,
          patience    = 300, 
          max_epochs  = 2000,
          eval_metric = ['rmse'])

# Terminate clock
#
stop = time.time()

print('[INFO] Time %.2f' % (stop - start))

## Evaluation

### Get predictions

In [ ]:
# Get predictions
#
pred = model.predict( testX )


# # Get prediction of each component tree
# #
# predictions = []
# for Tree in model.estimators_:
#     predictions += [ Tree.predict( testX ) ]
    
# predictions = np.array( predictions )

### Apply inverse scaling/transformation

In [ ]:
# Apply inverse scaling
#
for i in range( Horizon ):
    testY[:,  i] = scaler.inverse_transform( testY[:,  i].reshape(-1,1) ).squeeze(-1)
    pred[:, i]   = scaler.inverse_transform( pred[:, i].reshape(-1,1) ).squeeze(-1)


# Apply inverse transformation   
#
if (Transformation == True):
    testY = np.exp( testY ) - VALUE
    pred = np.exp( pred )   - VALUE

### Calculate Performance on Testing set - Prediction visualization


In [ ]:
print('[INFO] Feature: ', targetSeries)
print('------------------------------------------------')
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( Horizon ):

    Prices = pd.DataFrame([])        

    Prices[targetSeries] = testY[:,i]
    Prices['Prediction'] = pred[:,i]


    # Evaluation
    #
    MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )

    # Store results
    #
    Performance_Foresting_Model['RMSE']    += [ RMSE    ]
    Performance_Foresting_Model['MAE']     += [ MAE     ]
    Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
    Performance_Foresting_Model['R2']      += [ R2      ]

#     # Present results
#     #
#     print('Horizon: ', i)
#     print('> MAE:   ', MAE)
#     print('> RMSE:  ', RMSE)
#     print('> SMAPE: ', SMAPE)
#     print('> R2:    ', R2)
#     print()
    
    print('Horizon: %2i MAE %5.2f SMAPE: %5.2f R2: %.2f' %(i+1, MAE, SMAPE, R2) )

## Visualization

In [ ]:
print('[INFO] Feature: ', targetSeries)
print('------------------------------------------------')
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( Horizon ):

    Prices = pd.DataFrame([])        

    Prices[targetSeries] = testY[:,i]
    Prices['Prediction'] = pred[:,i]
            
            
    # Plot Real & Predicted values
    #
    Prices[:100].plot( figsize = (20, 3), marker = 'o' )
    #
    plt.title('Feature: {} - Horizon = {}'.format(targetSeries, i+1))
    plt.legend( frameon = False, fontsize = 14)
    plt.xticks(size = 12)
    plt.yticks(size = 12)
    plt.show()        

## Examples

In [ ]:
subplots = [331, 332, 333, 334, 335, 336,  337, 338, 339]
plt.figure( figsize = (20, 8) )
RandomInstances = [113, 34, 141, 325, 139, 185, 188, 27, 31]


for plot_id, i in enumerate(RandomInstances):

    plt.subplot(subplots[plot_id])
    #
    plt.plot(range(Lag, Lag + Horizon), testY[i], color='g', marker = 'o', linewidth = 2)
    plt.plot(range(Lag, Lag + Horizon), pred[i],  color='r', marker = 'o', linewidth = 2)

    plt.legend(['Instance', 'Future values', 'Prediction'], frameon = False, fontsize = 12)
plt.show()

In [ ]:
subplots = [331, 332, 333, 334, 335, 336,  337, 338, 339]
plt.figure( figsize = (20, 8) )
RandomInstances = [random.randint(1, testY.shape[0]) for i in range(0, 9)]

for plot_id, i in enumerate(RandomInstances):

    plt.subplot(subplots[plot_id])
    plt.grid()
#     plot_scatter(range(0, Lag),             testX[i,:Lag], color='b')
    plt.plot(range(Lag, Lag + Horizon), testY[i], color='g', marker = 'o', linewidth = 2)
    plt.plot(range(Lag, Lag + Horizon), pred[i],  color='r', marker = 'o', linewidth = 2)

    plt.legend(['Instance', 'Future values', 'Prediction'], frameon = False, fontsize = 12)
#     plt.ylim([0, 8])
plt.show()

# Store errors

In [ ]:
Outputs = pd.DataFrame([])
#
#
Outputs[targetSeries] = testY.flatten()
Outputs['TabNet']         = pred.flatten()
#
Outputs.to_csv('Predictions/TabNet.csv')